In [20]:
def arrays_equal(arr1, arr2):
    if len(arr1) != len(arr2):
        return False
    for i,v in enumerate(arr1):
        if v != arr2[i]:
            return False
    return True

In [21]:
def count_subsequents(line, subsequents):
    if not (len(line)):
        return []
    #possible_arrangments = [None] * len(line)
    sequences = []
    current_sequence = 0
    for i, char in enumerate(line):
        if char == "#":
            current_sequence += 1
        elif char == ".":
            sequences.append(current_sequence)
            current_sequence = 0
        else:
            sequence_length = len(sequences)
            if not arrays_equal(subsequents[:sequence_length], sequences):
                # only possible is to extend
                current_sequence += 1
            else:
                # inserted .
                seq1 = count_subsequents(line[i:], subsequents[sequence_length:])
                # inserted #
                updated_length = [subsequents[sequence_length] - current_sequence] + subsequents[sequence_length+1:]
                seq2 = count_subsequents(line[i], updated_length)
                results = []
                for new_seq in seq1+seq2:
                    if arrays_equal(subsequents, new_seq):
                        results.append(new_seq)
                return results
    if current_sequence:
        sequences.append(current_sequence) 
    return [sequences]

In [22]:
def place_subsequents2(line, subsequents):
    if not len(subsequents):
        if "#" in line:
            return 0
        else:
            return 1
    if subsequents[0] == 0:
        return place_subsequents(line, subsequents[1:])
    if len(line): 
        if line[0] == '.':
            return place_subsequents(line[1:], subsequents)
        elif line[0] == '#':
                return place_subsequents(line[1:], [subsequents[0]-1]+subsequents[1:])
    dot_counts = len([char for char in line if char in [".", "?"]])
    if dot_counts < len(subsequents)-1:
        return 0   
    current_continuos = subsequents[0]
    first_dot = line.find(".")
    if first_dot < 0:
        first_dot = len(line)
    if first_dot < current_continuos-1:
        return 0
    result = 0
    for i in range(first_dot - current_continuos+1):
        if line[i+current_continuos] == "#":
            continue 
        result += place_subsequents(line[i+current_continuos+1:], subsequents[1:])
    return result

In [23]:
def check_position(line, subsequents, positions):
    eligible_positions = [k+pos for i, pos in enumerate(positions) for k in range(subsequents[i])]
    eligible_positions_set = set(eligible_positions)
    if len(eligible_positions_set) != len(eligible_positions):
        return False
    # positions need to be at least one further away
    chains = 0    
    for i, eligible in enumerate(eligible_positions):
        if i == 0:
            chains += 1
        elif eligible - 1 != eligible_positions[i-1]:
            chains +=1
    if chains != len(positions):
        return False
    for i, char in enumerate(line):
        if char == "." and i in eligible_positions_set:
            return False
        elif char == '#' and i not in eligible_positions_set:
            return False
    return True

In [24]:
from itertools import combinations


def place_subsequents(line, subsequents):
    if not len(subsequents):
        if "#" in line:
            return 0
        else:
            return 1
    if line[-1] == ".":
        return place_subsequents(line[:-1], subsequents)
    if subsequents[0] == 0:
        return place_subsequents(line, subsequents[1:])
    last_possible_start = len(line) - subsequents[-1]
    #  .# 1
    # lp 1
    positions_to_consider = [position for position, char in enumerate(line) if char != '.' and position <= last_possible_start]
    amount_of_positions = len(subsequents)
    res = 0
    goodpositions = []
    for combo in combinations(positions_to_consider, amount_of_positions):
        if check_position(line, subsequents, combo):
            res+=1
    return res      
   

In [25]:
check_position("?#?#?#?#?#?#?#?", [1,3,1,6], [0,2,6,8])

False

In [26]:
check_position("?#?#?#?#?#?#?#?", [1,3,1,6], [1,3,7,9])

True

In [27]:
#.#.### 1,1,3
place_subsequents("#.#.###", [1, 1, 3])

1

In [28]:
place_subsequents("???.###", [1, 1, 3])

1

???.### 1,1,3 - 1 arrangement
.??..??...?##. 1,1,3 - 4 arrangements
?#?#?#?#?#?#?#? 1,3,1,6 - 1 arrangement
????.#...#... 4,1,1 - 1 arrangement
????.######..#####. 1,6,5 - 4 arrangements
?###???????? 3,2,1 - 10 arrangements

In [29]:
place_subsequents(".??..??...?##." ,[1,1,3])

4

In [30]:
place_subsequents("?#?#?#?#?#?#?#?", [1,3,1,6])

1

In [31]:
place_subsequents("????.#...#..." ,[4,1,1])

1

In [32]:
place_subsequents("????.######..#####." ,[1,6,5])

4

In [33]:
place_subsequents("?###????????" ,[3,2,1])

10

In [34]:
place_subsequents("??#????????????#", [1,2,1,2,1])

41

In [38]:
def compute_next(line):
    #print(line)
    left, right = line.split()
    return place_subsequents(left, [int(x) for x in right.split(",")])

In [39]:
with open("day12.txt") as f:
    lines = f.readlines()
results = []
for line in lines:
    results.append(compute_next(line))
sum(results)

6871